<a href="https://colab.research.google.com/github/ishikasingh/Affective-text-gen/blob/master/AffectiveTextGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#### Code based on PPLM framework ####
!pip install transformers

In [17]:
import argparse
import json
from operator import add
from typing import List, Optional, Tuple, Union

import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import trange
from transformers import GPT2Tokenizer
from transformers.file_utils import cached_path
from transformers import GPT2LMHeadModel
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [18]:
class ClassificationHead(torch.nn.Module):
    """Classification Head for  transformer encoders"""

    def __init__(self, class_size, embed_size):
        super(ClassificationHead, self).__init__()
        self.class_size = class_size
        self.embed_size = embed_size
        # self.mlp1 = torch.nn.Linear(embed_size, embed_size)
        # self.mlp2 = (torch.nn.Linear(embed_size, class_size))
        self.mlp = torch.nn.Linear(embed_size, class_size)

    def forward(self, hidden_state):
        # hidden_state = F.relu(self.mlp1(hidden_state))
        # hidden_state = self.mlp2(hidden_state)
        logits = self.mlp(hidden_state)
        return logits

In [19]:
PPLM_BOW = 1
PPLM_DISCRIM = 2
PPLM_BOW_DISCRIM = 3
BOW_AFFECT = 4
SMALL_CONST = 1e-15
BIG_CONST = 1e10

QUIET = 0
REGULAR = 1
VERBOSE = 2
VERY_VERBOSE = 3
VERBOSITY_LEVELS = {
    'quiet': QUIET,
    'regular': REGULAR,
    'verbose': VERBOSE,
    'very_verbose': VERY_VERBOSE,
}

BAG_OF_WORDS_ARCHIVE_MAP = {
    'legal': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/legal.txt",
    'military': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/military.txt",
    'monsters': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/monsters.txt",
    'politics': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/politics.txt",
    'positive_words': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/positive_words.txt",
    'religion': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/religion.txt",
    'science': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/science.txt",
    'space': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/space.txt",
    'technology': "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/bow/technology.txt",
}

DISCRIMINATOR_MODELS_PARAMS = {
    "clickbait": {
        "url": "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/discriminators/clickbait_classifier_head.pt",
        "class_size": 2,
        "embed_size": 1024,
        "class_vocab": {"non_clickbait": 0, "clickbait": 1},
        "default_class": 1,
        "pretrained_model": "gpt2-medium",
    },
    "sentiment": {
        "url": "https://s3.amazonaws.com/models.huggingface.co/bert/pplm/discriminators/SST_classifier_head.pt",
        "class_size": 5,
        "embed_size": 1024,
        "class_vocab": {"very_positive": 2, "very_negative": 3},
        "default_class": 3,
        "pretrained_model": "gpt2-medium",
    },
}

In [20]:
def to_var(x, requires_grad=False, volatile=False, device='cuda'):
    if torch.cuda.is_available() and device == 'cuda':
        x = x.cuda()
    elif device != 'cuda':
        x = x.to(device)
    return Variable(x, requires_grad=requires_grad, volatile=volatile)


def top_k_filter(logits, k, probs=False):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        if probs:
            return torch.where(logits < batch_mins,
                               torch.ones_like(logits) * 0.0, logits)
        return torch.where(logits < batch_mins,
                           torch.ones_like(logits) * -BIG_CONST,
                           logits)

def gaussian(x, mu, sig):
  x = np.array(x)
  return list(np.exp(-0.5*((x-mu)/sig)**2)/(sig*(2*np.pi)**0.5))

def perturb_past(
        past,
        model,
        last,
        affect_weight=0.2,
        unpert_past=None,
        unpert_logits=None,
        accumulated_hidden=None,
        grad_norms=None,
        stepsize=0.01,
        one_hot_bows_vectors=None,
        one_hot_bows_affect=None,
        affect_int = None,
        knob = None,
        classifier=None,
        class_label=None,
        loss_type=0,
        num_iterations=3,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        kl_scale=0.01,
        device='cuda',
        verbosity_level=REGULAR
):
    # Generate inital perturbed past
    grad_accumulator = [
        (np.zeros(p.shape).astype("float32"))
        for p in past
    ]
    
    if accumulated_hidden is None:
        accumulated_hidden = 0

    if decay:
        decay_mask = torch.arange(
            0.,
            1.0 + SMALL_CONST,
            1.0 / (window_length)
        )[1:]
    else:
        decay_mask = 1.0

    # TODO fix this comment (SUMANTH)
    # Generate a mask is gradient perturbated is based on a past window
    _, _, _, curr_length, _ = past[0].shape

    if curr_length > window_length and window_length > 0:
        ones_key_val_shape = (
                tuple(past[0].shape[:-2])
                + tuple([window_length])
                + tuple(past[0].shape[-1:])
        )

        zeros_key_val_shape = (
                tuple(past[0].shape[:-2])
                + tuple([curr_length - window_length])
                + tuple(past[0].shape[-1:])
        )

        ones_mask = torch.ones(ones_key_val_shape)
        ones_mask = decay_mask * ones_mask.permute(0, 1, 2, 4, 3)
        ones_mask = ones_mask.permute(0, 1, 2, 4, 3)

        window_mask = torch.cat(
            (ones_mask, torch.zeros(zeros_key_val_shape)),
            dim=-2
        ).to(device)
    else:
        window_mask = torch.ones_like(past[0]).to(device)

    # accumulate perturbations for num_iterations
    loss_per_iter = []
    new_accumulated_hidden = None
    for i in range(num_iterations):
        if verbosity_level >= VERBOSE:
            print("Iteration ", i + 1)
        curr_perturbation = [
            to_var(torch.from_numpy(p_), requires_grad=True, device=device)
            for p_ in grad_accumulator
        ]

        # Compute hidden using perturbed past
        perturbed_past = list(map(add, past, curr_perturbation))
        _, _, _, curr_length, _ = curr_perturbation[0].shape
        all_logits, _, all_hidden = model(last, past=perturbed_past)
        hidden = all_hidden[-1]
        new_accumulated_hidden = accumulated_hidden + torch.sum(
            hidden,
            dim=1
        ).detach()
        # TODO: Check the layer-norm consistency of this with trained discriminator (Sumanth)
        logits = all_logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)

        loss = 0.0
        loss_list = []
        if loss_type == PPLM_BOW or loss_type == BOW_AFFECT:
            for one_hot_bow in one_hot_bows_vectors:
                bow_logits = torch.mm(probs, torch.t(one_hot_bow))
                #print(type(bow_logits))
                bow_loss = -torch.log(torch.sum(bow_logits))
                #print(bow_loss)
                loss +=  bow_loss
                loss_list.append(bow_loss)
            if loss_type == BOW_AFFECT:
              for one_hot_bow in one_hot_bows_affect:
                  bow_logits = torch.mm(probs, torch.t(one_hot_bow))
                 # print(bow_logits.size(), torch.FloatTensor(affect_int).size())
                  bow_loss = -torch.log(torch.matmul(bow_logits, torch.t(torch.FloatTensor(gaussian(affect_int, knob, .1)).to(device))))#-torch.log(torch.sum(bow_logits))#
                  # print(bow_loss)

                  loss += affect_weight * bow_loss[0]
                  loss_list.append(bow_loss)
            if verbosity_level >= VERY_VERBOSE:
                print(" pplm_bow_loss:", loss.data.cpu().numpy())

        kl_loss = 0.0
        if kl_scale > 0.0:
            unpert_probs = F.softmax(unpert_logits[:, -1, :], dim=-1)
            unpert_probs = (
                    unpert_probs + SMALL_CONST *
                    (unpert_probs <= SMALL_CONST).float().to(device).detach()
            )
            correction = SMALL_CONST * (probs <= SMALL_CONST).float().to(
                device).detach()
            corrected_probs = probs + correction.detach()
            kl_loss = kl_scale * (
                (corrected_probs * (corrected_probs / unpert_probs).log()).sum()
            )
            if verbosity_level >= VERY_VERBOSE:
                print(' kl_loss', kl_loss.data.cpu().numpy())
            loss += kl_loss

        loss_per_iter.append(loss.data.cpu().numpy())
        if verbosity_level >= VERBOSE:
            print(' pplm_loss', (loss - kl_loss).data.cpu().numpy())

        # compute gradients
        loss.backward()

        # calculate gradient norms
        if grad_norms is not None and loss_type == PPLM_BOW:
            grad_norms = [
                torch.max(grad_norms[index], torch.norm(p_.grad * window_mask))
                for index, p_ in enumerate(curr_perturbation)
            ]
        else:
            grad_norms = [
                (torch.norm(p_.grad * window_mask) + SMALL_CONST)
                for index, p_ in enumerate(curr_perturbation)
            ]

        # normalize gradients
        grad = [
            -stepsize *
            (p_.grad * window_mask / grad_norms[
                index] ** gamma).data.cpu().numpy()
            for index, p_ in enumerate(curr_perturbation)
        ]

        # accumulate gradient
        grad_accumulator = list(map(add, grad, grad_accumulator))

        # reset gradients, just to make sure
        for p_ in curr_perturbation:
            p_.grad.data.zero_()

        # removing past from the graph
        new_past = []
        for p_ in past:
            new_past.append(p_.detach())
        past = new_past

    # apply the accumulated perturbations to the past
    grad_accumulator = [
        to_var(torch.from_numpy(p_), requires_grad=True, device=device)
        for p_ in grad_accumulator
    ]
    pert_past = list(map(add, past, grad_accumulator))

    return pert_past, new_accumulated_hidden, grad_norms, loss_per_iter


def get_classifier(
        name: Optional[str],
        class_label: Union[str, int],
        device: str,
        verbosity_level: int = REGULAR
) -> Tuple[Optional[ClassificationHead], Optional[int]]:
    if name is None:
        return None, None

    params = DISCRIMINATOR_MODELS_PARAMS[name]
    classifier = ClassificationHead(
        class_size=params['class_size'],
        embed_size=params['embed_size']
    ).to(device)
    if "url" in params:
        resolved_archive_file = cached_path(params["url"])
    elif "path" in params:
        resolved_archive_file = params["path"]
    else:
        raise ValueError("Either url or path have to be specified "
                         "in the discriminator model parameters")
    classifier.load_state_dict(
        torch.load(resolved_archive_file, map_location=device))
    classifier.eval()

    if isinstance(class_label, str):
        if class_label in params["class_vocab"]:
            label_id = params["class_vocab"][class_label]
        else:
            label_id = params["default_class"]
            if verbosity_level >= REGULAR:
                print("class_label {} not in class_vocab".format(class_label))
                print("available values are: {}".format(params["class_vocab"]))
                print("using default class {}".format(label_id))

    elif isinstance(class_label, int):
        if class_label in set(params["class_vocab"].values()):
            label_id = class_label
        else:
            label_id = params["default_class"]
            if verbosity_level >= REGULAR:
                print("class_label {} not in class_vocab".format(class_label))
                print("available values are: {}".format(params["class_vocab"]))
                print("using default class {}".format(label_id))

    else:
        label_id = params["default_class"]

    return classifier, label_id


def get_bag_of_words_indices(bag_of_words_ids_or_paths: List[str], tokenizer) -> \
        List[List[List[int]]]:
    bow_indices = []
    for id_or_path in bag_of_words_ids_or_paths:
        if id_or_path in BAG_OF_WORDS_ARCHIVE_MAP:
            filepath = cached_path(BAG_OF_WORDS_ARCHIVE_MAP[id_or_path])
        else:
            filepath = id_or_path
        with open(filepath, "r") as f:
            words = f.read().strip().split("\n")
        bow_indices.append(
            [tokenizer.encode(word.strip(),
                              add_prefix_space=True,
                              add_special_tokens=False)
             for word in words])
    return bow_indices

def get_affect_words_and_int (affect_class):
  emotions = "https://raw.githubusercontent.com/ishikasingh/Affective-text-gen/master/NRC-Emotion-Intensity-Lexicon-v1.txt"
  filepath = cached_path(emotions)
  with open(filepath, "r") as f:
      words = f.read().strip().split("\n")[1:]
  words = [w.split("\t") for w in words]
  return [w[0] for w in words if w[1] == affect_class], [float(w[-1]) for w in words if w[1] == affect_class]

def build_bows_one_hot_vectors(bow_indices, tokenizer, device='cuda'):
    if bow_indices is None:
        return None

    one_hot_bows_vectors = []
    for single_bow in bow_indices:
        single_bow = list(filter(lambda x: len(x) <= 1, single_bow))

        single_bow = torch.tensor(single_bow).to(device)
        num_words = single_bow.shape[0]
        # print(num_words)
        one_hot_bow = torch.zeros(num_words, tokenizer.vocab_size).to(device)
        one_hot_bow.scatter_(1, single_bow, 1)
        one_hot_bows_vectors.append(one_hot_bow)
    return one_hot_bows_vectors

def build_bows_one_hot_vectors_aff(bow_indices,affect_int, tokenizer, device='cuda'):
    if bow_indices is None or affect_int is None:
        return None, None

    one_hot_bows_vectors = []
    # print(np.array(bow_indices).shape)
    for single_bow in bow_indices:
        zipped = [[single_bow[i], affect_int[i]] for i in range(len(single_bow))]
        single_bow_int = list(filter(lambda x: len(x[0]) <= 1, zipped))
        single_bow = [single_bow_int[i][0] for i in range(len(single_bow_int)) ]
        affect_ints = [single_bow_int[i][1] for i in range(len(single_bow_int)) ]
        # print(single_bow, affect_ints)
        # print(len(single_bow), len(affect_ints))
        single_bow = torch.tensor(single_bow).to(device)
        num_words = single_bow.shape[0]
        # print(num_words)
        one_hot_bow = torch.zeros(num_words, tokenizer.vocab_size).to(device)
        one_hot_bow.scatter_(1, single_bow, 1)
        one_hot_bows_vectors.append(one_hot_bow)
    return one_hot_bows_vectors, affect_ints



def full_text_generation(
        model,
        tokenizer,
        affect_weight=0.2,
        knob = None,
        context=None,
        num_samples=1,
        device="cuda",
        bag_of_words=None,
        bag_of_words_affect=None,
        discrim=None,
        class_label=None,
        length=100,
        stepsize=0.02,
        temperature=1.0,
        top_k=10,
        sample=True,
        num_iterations=3,
        grad_length=10000,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        gm_scale=0.9,
        kl_scale=0.01,
        verbosity_level=REGULAR,
        **kwargs
):
    classifier, class_id = get_classifier(discrim, class_label, device)
    # print("bog is here", bag_of_words)
    # print("affect is: ", bag_of_words_affect)
    bow_indices = []
    bow_indices_affect = []
    if bag_of_words:
      bow_indices = get_bag_of_words_indices(bag_of_words.split(";"), tokenizer)
    if bag_of_words_affect: 
      affect_words, affect_int = get_affect_words_and_int(bag_of_words_affect)
      bow_indices_affect.append([tokenizer.encode(word.strip(),add_prefix_space=True, add_special_tokens=False)for word in affect_words])
    # print("aff1", affect_int)
    loss_type = PPLM_BOW
    if bag_of_words_affect:
      loss_type = BOW_AFFECT

    unpert_gen_tok_text, _, _ = generate_text_pplm(
        model=model,
        tokenizer=tokenizer,
        context=context,
        device=device,
        length=length,
        sample=sample,
        perturb=False,
        verbosity_level=verbosity_level
    )

    if device == 'cuda':
        torch.cuda.empty_cache()

    pert_gen_tok_texts = []
    discrim_losses = []
    losses_in_time = []
    print("After Perturbation")
    for i in range(num_samples):
        pert_gen_tok_text, discrim_loss, loss_in_time = generate_text_pplm(
            model=model,
            tokenizer=tokenizer,
            affect_weight=affect_weight,
            context=context,
            device=device,
            perturb=True,
            bow_indices=bow_indices,
            bow_indices_affect=bow_indices_affect,
            affect_int = affect_int,
            knob = knob,
            classifier=classifier,
            class_label=class_id,
            loss_type=loss_type,
            length=length,
            stepsize=stepsize,
            temperature=temperature,
            top_k=top_k,
            sample=sample,
            num_iterations=num_iterations,
            grad_length=grad_length,
            horizon_length=horizon_length,
            window_length=window_length,
            decay=decay,
            gamma=gamma,
            gm_scale=gm_scale,
            kl_scale=kl_scale,
            verbosity_level=verbosity_level
        )
        pert_gen_tok_texts.append(pert_gen_tok_text)
        if classifier is not None:
            discrim_losses.append(discrim_loss.data.cpu().numpy())
        losses_in_time.append(loss_in_time)

    if device == 'cuda':
        torch.cuda.empty_cache()

    return unpert_gen_tok_text, pert_gen_tok_texts, discrim_losses, losses_in_time




In [21]:
def get_affect_words_and_int1 (affect_class):
  emotions = "https://raw.githubusercontent.com/ishikasingh/Affective-text-gen/master/NRC-AffectIntensity-Lexicon.txt"
  filepath = cached_path(emotions)
  with open(filepath, "r") as f:
      words = f.read().strip().split("\n")[37:]
  words = [w.split("\t") for w in words]
  return [w[0] for w in words if w[-1] == affect_class], [float(w[1]) for w in words if w[-1] == affect_class]

In [22]:
k, j = get_affect_words_and_int ("anger")
k1, j1 = get_affect_words_and_int1 ("anger")

# type(torch.matmul(torch.FloatTensor(j[:10]), torch.FloatTensor(j[:10])))

In [23]:
j==j1, k==k1
k[:5], k1[:5], j[:5], j1[:5]

(['outraged', 'brutality', 'hatred', 'hateful', 'terrorize'],
 ['outraged', 'brutality', 'hatred', 'hateful', 'terrorize'],
 [0.964, 0.959, 0.953, 0.94, 0.939],
 [0.964, 0.959, 0.953, 0.94, 0.939])

In [24]:
def run_pplm_example(
        pretrained_model="gpt2-medium",
        cond_text="",
        affect_weight=0.2,
        knob = None,
        uncond=False,
        num_samples=1,
        bag_of_words=None,
        bag_of_words_affect=None,
        discrim=None,
        discrim_weights=None,
        discrim_meta=None,
        class_label=-1,
        length=100,
        stepsize=0.02,
        temperature=1.0,
        top_k=10,
        sample=True,
        num_iterations=3,
        grad_length=10000,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        gm_scale=0.9,
        kl_scale=0.01,
        seed=0,
        no_cuda=False,
        colorama=False,
        verbosity='regular'
):
    # set Random seed
    torch.manual_seed(seed)
    np.random.seed(seed)

    # set verbosiry
    verbosity_level = VERBOSITY_LEVELS.get(verbosity.lower(), REGULAR)

    # # set the device
    # device = "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"

    if discrim == 'generic':
        set_generic_model_params(discrim_weights, discrim_meta)

    if discrim is not None:
        discriminator_pretrained_model = DISCRIMINATOR_MODELS_PARAMS[discrim][
            "pretrained_model"
        ]
        if pretrained_model != discriminator_pretrained_model:
            pretrained_model = discriminator_pretrained_model
            if verbosity_level >= REGULAR:
                print("discrim = {}, pretrained_model set "
                "to discriminator's = {}".format(discrim, pretrained_model))

    # # load pretrained model
    # model = GPT2LMHeadModel.from_pretrained(
    #     pretrained_model,
    #     output_hidden_states=True
    # )
    # model.to(device)
    # model.eval()

    # # load tokenizer
    # tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)

    # Freeze GPT-2 weights
    for param in model.parameters():
        param.requires_grad = False

    # figure out conditioning text
    if uncond:
        tokenized_cond_text = tokenizer.encode([tokenizer.bos_token],add_special_tokens=False)
    else:
        raw_text = cond_text
        while not raw_text:
            print("Did you forget to add `--cond_text`? ")
            raw_text = input("Model prompt >>> ")
        tokenized_cond_text = tokenizer.encode(tokenizer.bos_token + raw_text,add_special_tokens=False)
    print("= Prefix of sentence =")
    # generate unperturbed and perturbed texts

    # full_text_generation returns:
    # unpert_gen_tok_text, pert_gen_tok_texts, discrim_losses, losses_in_time
    unpert_gen_tok_text, pert_gen_tok_texts, _, _ = full_text_generation(
        model=model,
        tokenizer=tokenizer,
        affect_weight=affect_weight,
        knob = knob,
        context=tokenized_cond_text,
        device=device,
        num_samples=num_samples,
        bag_of_words=bag_of_words,
        bag_of_words_affect=bag_of_words_affect,
        discrim=discrim,
        class_label=class_label,
        length=length,
        stepsize=stepsize,
        temperature=temperature,
        top_k=top_k,
        sample=sample,
        num_iterations=num_iterations,
        grad_length=grad_length,
        horizon_length=horizon_length,
        window_length=window_length,
        decay=decay,
        gamma=gamma,
        gm_scale=gm_scale,
        kl_scale=kl_scale,
        verbosity_level=verbosity_level
    )

    # untokenize unperturbed text
    unpert_gen_text = tokenizer.decode(unpert_gen_tok_text.tolist()[0])

    if verbosity_level >= REGULAR:
        print("=" * 80)
    print("= Unperturbed generated text =")
    print(unpert_gen_text)
    print()

    generated_texts = []

    # iterate through the perturbed texts
    for i, pert_gen_tok_text in enumerate(pert_gen_tok_texts):
        try:
            # untokenize unperturbed text
            pert_gen_text = tokenizer.decode(pert_gen_tok_text.tolist()[0])
            print("= Perturbed generated text {} =".format(i + 1))
            print(pert_gen_text)
            print()
        except:
            pass

        # keep the prefix, perturbed seq, original seq for each index
        generated_texts.append(
            (tokenized_cond_text, pert_gen_tok_text, unpert_gen_tok_text)
        )

    return

In [25]:
def generate_text_pplm(
        model,
        tokenizer,
        affect_weight=0.2,
        context=None,
        past=None,
        device="cuda",
        perturb=True,
        bow_indices=None,
        bow_indices_affect=None,
        affect_int = None,
        knob = None,
        classifier=None,
        class_label=None,
        loss_type=0,
        length=100,
        stepsize=0.02,
        temperature=1.0,
        top_k=10,
        sample=True,
        num_iterations=3,
        grad_length=10000,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        gm_scale=0.9,
        kl_scale=0.01,
        verbosity_level=REGULAR
):
    output_so_far = None
    if context:
        context_t = torch.tensor(context, device=device, dtype=torch.long)
        while len(context_t.shape) < 2:
            context_t = context_t.unsqueeze(0)
        output_so_far = context_t

    # collect one hot vectors for bags of words
    one_hot_bows_vectors = build_bows_one_hot_vectors(bow_indices, tokenizer, device)
    affect_int_orig = affect_int
    one_hot_bows_affect, affect_int = build_bows_one_hot_vectors_aff(bow_indices_affect, affect_int, tokenizer, device)
#    print(torch.FloatTensor(one_hot_bows_affect).size())
    grad_norms = None
    last = None
    unpert_discrim_loss = 0
    loss_in_time = []

    if verbosity_level >= VERBOSE:
        range_func = trange(length, ascii=True)
    else:
        range_func = range(length)
    count = 0
    int_score = 0
    for i in range_func:
        if count == 3:
          break
        # Get past/probs for current output, except for last word
        # Note that GPT takes 2 inputs: past + current_token

        # run model forward to obtain unperturbed
        if past is None and output_so_far is not None:
            last = output_so_far[:, -1:]
            if output_so_far.shape[1] > 1:
                _, past, _ = model(output_so_far[:, :-1])

        unpert_logits, unpert_past, unpert_all_hidden = model(output_so_far)
        unpert_last_hidden = unpert_all_hidden[-1]

        # check if we are abowe grad max length
        if i >= grad_length:
            current_stepsize = stepsize * 0
        else:
            current_stepsize = stepsize

        # modify the past if necessary
        if not perturb or num_iterations == 0:
            pert_past = past

        else:
            accumulated_hidden = unpert_last_hidden[:, :-1, :]
            accumulated_hidden = torch.sum(accumulated_hidden, dim=1)

            if past is not None:
                pert_past, _, grad_norms, loss_this_iter = perturb_past(
                    past,
                    model,
                    last,
                    affect_weight = affect_weight,
                    unpert_past=unpert_past,
                    unpert_logits=unpert_logits,
                    accumulated_hidden=accumulated_hidden,
                    grad_norms=grad_norms,
                    stepsize=current_stepsize,
                    one_hot_bows_vectors=one_hot_bows_vectors,
                    one_hot_bows_affect=one_hot_bows_affect,
                    affect_int = affect_int,
                    knob = knob,
                    classifier=classifier,
                    class_label=class_label,
                    loss_type=loss_type,
                    num_iterations=num_iterations,
                    horizon_length=horizon_length,
                    window_length=window_length,
                    decay=decay,
                    gamma=gamma,
                    kl_scale=kl_scale,
                    device=device,
                    verbosity_level=verbosity_level
                )
                loss_in_time.append(loss_this_iter)
            else:
                pert_past = past

        pert_logits, past, pert_all_hidden = model(last, past_key_values=pert_past)
        pert_logits = pert_logits[:, -1, :] / temperature  # + SMALL_CONST
        pert_probs = F.softmax(pert_logits, dim=-1)

        if classifier is not None:
            ce_loss = torch.nn.CrossEntropyLoss()
            prediction = classifier(torch.mean(unpert_last_hidden, dim=1))
            label = torch.tensor([class_label], device=device,
                                 dtype=torch.long)
            unpert_discrim_loss = ce_loss(prediction, label)
            if verbosity_level >= VERBOSE:
                print(
                    "unperturbed discrim loss",
                    unpert_discrim_loss.data.cpu().numpy()
                )
        else:
            unpert_discrim_loss = 0

        # Fuse the modified model and original model
        if perturb:

            unpert_probs = F.softmax(unpert_logits[:, -1, :], dim=-1)

            pert_probs = ((pert_probs ** gm_scale) * (
                    unpert_probs ** (1 - gm_scale)))  # + SMALL_CONST
            pert_probs = top_k_filter(pert_probs, k=top_k,
                                      probs=True)  # + SMALL_CONST

            # rescale
            if torch.sum(pert_probs) <= 1:
                pert_probs = pert_probs / torch.sum(pert_probs)

        else:
            pert_logits = top_k_filter(pert_logits, k=top_k)  # + SMALL_CONST
            pert_probs = F.softmax(pert_logits, dim=-1)

        # sample or greedy
        if sample:
            last = torch.multinomial(pert_probs, num_samples=1)
            # print('pert_prob, last ', pert_probs, last)

        else:
            _, last = torch.topk(pert_probs, k=1, dim=-1)

        # update context/output_so_far appending the new token
        output_so_far = (
            last if output_so_far is None
            else torch.cat((output_so_far, last), dim=1)
        )
        if verbosity_level >= REGULAR:
            print(tokenizer.decode(output_so_far.tolist()[0]))
        if(tokenizer.decode(output_so_far.tolist()[0])[-1] == '.' ):
          count = count+1
        if bow_indices_affect is not None and [output_so_far.tolist()[0][-1]] in bow_indices_affect[0]:
          int_word = affect_int_orig[bow_indices_affect[0].index([output_so_far.tolist()[0][-1]])]
          print(tokenizer.decode(output_so_far.tolist()[0][-1]), int_word)
          int_score = int_score + int_word
    print("int_score: ", int_score)
    # print("int.. " , output_so_far.tolist()[0][-1])
    return output_so_far, unpert_discrim_loss, loss_in_time


def set_generic_model_params(discrim_weights, discrim_meta):
    if discrim_weights is None:
        raise ValueError('When using a generic discriminator, '
                         'discrim_weights need to be specified')
    if discrim_meta is None:
        raise ValueError('When using a generic discriminator, '
                         'discrim_meta need to be specified')

    with open(discrim_meta, 'r') as discrim_meta_file:
        meta = json.load(discrim_meta_file)
    meta['path'] = discrim_weights
    DISCRIMINATOR_MODELS_PARAMS['generic'] = meta

In [26]:
# set the device
if torch.cuda.is_available():
    device = "cuda" #if torch.cuda.is_available() and not no_cuda else "cpu"
else:
    device = "cpu"
    
 # load pretrained model
pretrained_model="gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(
    pretrained_model,
    output_hidden_states=True
)
model.to(device)
model.eval()

# load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)

# Run the Model

In [27]:
def f(Knob, Prompt, Topic, Affect):
    run_pplm_example(
          affect_weight=1,  # it is the convergence rate of affect loss, don't change it :-p
          knob = Knob, # 0-1, play with it as much as you want
          cond_text=Prompt,
          num_samples=1,
          bag_of_words=Topic,
          bag_of_words_affect=Affect,
          length=500,
          stepsize=0.01,
          sample=True,
          num_iterations=3,
          window_length=5,
          gamma=1.5,
          gm_scale=0.95,
          kl_scale=0.01,
          verbosity='quiet'
      )
def run():
    interact_manual(f, Knob=(0,1, 0.1), Prompt="There exists", \
         Topic = ['legal','military','monsters','politics','positive_words', 'religion', 'science','space','technology'], \
         Affect = ['fear', 'joy', 'anger', 'sadness', 'anticipation', 'disgust', 'surprise', 'trust']);

In [28]:
run()

interactive(children=(FloatSlider(value=0.0, description='Knob', max=1.0), Text(value='There exists', descript…

In [15]:
topics = ['legal']#,'military','monsters','politics','positive_words', 'religion', 'science','space','technology']
affects = ['fear']#, 'anger', 'sadness'] #'fear', 
knob_vals = [0.8]#,0.5,0.7,1]

for topic in topics:
  for affect in affects:
    for knob in knob_vals:
      print("topic:", topic, ", affect:", affect, ", knob is:", knob)
      run_pplm_example(
          affect_weight=1,  # it is the convergence rate of affect loss, don't change it :-p
          knob = knob, # 0-1, play with it as much as you want
          cond_text="It is",
          num_samples=1,
          bag_of_words=topic,
          bag_of_words_affect=affect,
          length=50,
          stepsize=0.005, #topic, affect convergence rate
          sample=True,
          num_iterations=10,
          window_length=5,
          gamma=1.5,
          gm_scale=0.95,
          kl_scale=0.01,
          verbosity='quiet'
      )

topic: legal , affect: fear , knob is: 0.8
= Prefix of sentence =


/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:746: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


int_score:  0
After Perturbation
 shooting 0.812
 rampage 0.781
 killing 0.906
 violence 0.812
 shot 0.734
int_score:  4.045
= Unperturbed generated text =
<|endoftext|>It is no secret that the United States is the most economically powerful nation in the world. However, it's important to realize how much more powerful we are when we consider our economic growth.

The United States has a population that is now greater than the

= Perturbed generated text 1 =
<|endoftext|>It is the first time in US history that two women from opposing religious sects have been killed in a US shooting rampage.

The killing in San Bernardino, California, of 14 people was sparked by a domestic violence dispute between the couple, who were shot dead



# Evaluate Perplexity

In [ ]:
!pip install pytorch_pretrained_bert


In [ ]:
import math
import torch
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
# Load pre-trained model (weights)
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')


# 21.31652459381952, 61.45907380241148, 26.24923942649312

In [ ]:
def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss=model(tensor_input, lm_labels=tensor_input)
    # print(sentence, tokenize_input, tensor_input, loss)
    return math.exp(loss)
    
# use your sentences that you want to evaluate

In [ ]:
a=['i am a girl . you are a boy .',
                'there is a plane on the desk',
                        "there 's a pen on the desk", "this is me <eos>", "hi i want... i want your book", "there is is some noise"]
print([score(i) for i in a])

In [ ]:
import pandas as pd
data = []
file_object = open('AffectLM_GenData.txt', 'r')
for line in file_object:
  data.append(line[:-1].replace('<eos>', '.').split(' #### '))
  data[-1].append(score(data[-1][-1]))
file_object.close()


In [ ]:
line = []
with open('pplm.txt','r') as f:
  for l in f.readlines():
    l = l.split('\t')
    l[-1] = l[-1][:-1]
    line.append(l)
    
line[0]

In [ ]:
for i in range(len(line)):
  line[i].append(score(line[i][2]))
line[:3]

In [ ]:
inten = [[] for i in range(6)]
for i in line:
  # if i[0] not in ['The book', 'The robots','The country', 'The issue focused on', 'The relationship','The road']:# and i[1] in task1emos:
  if i[0] not in emos:
    emos.append(i[1])
  if i[1] =='0.01':
    inten[0].append(i[3])
  if i[1] =='0.02':
    inten[1].append(i[3])
  if i[1] =='0.05':
    inten[2].append(i[3])
  if i[1] =='0.1':
    inten[3].append(i[3])
  if i[1] =='0.5':
    inten[4].append(i[3])
  if i[1] =='1':
    inten[5].append(i[3])


[[inten.index(i),sum(i)/len(i)] for i in inten]

In [ ]:
import pickle
with open('affLMdata_eval', 'wb') as f:
  pickle.dump(data, f)

In [ ]:
import pickle
with open('affLMdata_eval', 'rb') as f:
  data = pickle.load(f)
data[:5]

In [ ]:
data[10:14]

In [ ]:
inten = [[] for i in range(6)]
task1emos = ['joy', 'sadness', 'anger', 'posemo', 'sad', 'angry']
prompts = []
emo = []
for i in data:
  if i[0] not in ['The book', 'The robots','The country', 'The issue focused on', 'The relationship','The road', 'the book', 'the robots','the country', 'the issue focused on', 'the relationship','the road'] and i[1] in task1emos:
    if i[0] not in prompts:
      prompts.append(i[0])
    if i[1] not in emo:
      emo.append(i[1])
    if i[2] =='0.0':
      inten[0].append(i[4])
    if i[2] =='1.0':
      inten[1].append(i[4])
    if i[2] =='2.0':
      inten[2].append(i[4])
    if i[2] =='3.0':
      inten[3].append(i[4])
    if i[2] =='4.0':
      inten[4].append(i[4])
    if i[2] =='5.0':
      inten[5].append(i[4])


[[inten.index(i),sum(i)/len(i)] for i in inten], emo, prompts

In [ ]:
pr = []
for i in data:
  if i[0] not in pr:
    pr.append(i[0])

In [ ]:
[[] for i in range(2)]

In [ ]:
import pickle
with open('pplm_sentences_scores.jpg', 'rb') as f:
  data = pickle.load(f)
data[:5]

In [ ]:
# !touch pplm.txt

# Other Plottings and p-value Evaluations

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My\ Drive/NLP\ |\ AffTextGen\ |\ Human Eval

In [ ]:
def get_affect_words_and_int (affect_class):
  emotions = "https://raw.githubusercontent.com/ishikasingh/Affective-text-gen/master/NRC-Emotion-Intensity-Lexicon-v1.txt"
  filepath = cached_path(emotions)
  with open(filepath, "r") as f:
      words = f.read().strip().split("\n")[1:]
  words = [w.split("\t") for w in words]
  return [w[0] for w in words if w[1] == affect_class], [float(w[-1]) for w in words if w[1] == affect_class]

In [ ]:
joy = get_affect_words_and_int('joy')
anger = get_affect_words_and_int('anger')
sad = get_affect_words_and_int('sadness')
fear = get_affect_words_and_int('fear')
anticipation = get_affect_words_and_int('anticipation')
disgust = get_affect_words_and_int('disgust')
surprise = get_affect_words_and_int('surprise')
trust = get_affect_words_and_int('trust')

In [ ]:
i=11
j = [joy[1][i] for i in range(len(joy[0])) if i%100==0]
j1 = [joy[0][i] for i in range(len(joy[0])) if i%100==0]
a = [anger[1][i] for i in range(len(anger[0])) if i%(int(len(anger[0])/12))==0]
a1 = [anger[0][i] for i in range(len(anger[0])) if i%(int(len(anger[0])/12))==0]
s = [sad[1][i] for i in range(len(sad[0])) if i%(int(len(sad[0])/12))==0]
s1 = [sad[0][i] for i in range(len(sad[0])) if i%(int(len(sad[0])/12))==0]
f = [fear[1][i] for i in range(len(fear[0])) if i%(int(len(fear[0])/12))==0]
f1 = [fear[0][i] for i in range(len(fear[0])) if i%(int(len(fear[0])/12))==0]
an = [anticipation[1][i] for i in range(len(anticipation[0])) if i%(int(len(anticipation[0])/12))==0]
an1 = [anticipation[0][i] for i in range(len(anticipation[0])) if i%(int(len(anticipation[0])/12))==0]
d = [disgust[1][i] for i in range(len(disgust[0])) if i%(int(len(disgust[0])/12))==0 and disgust[0][i]!='pornography']
d1 = [disgust[0][i] for i in range(len(disgust[0])) if i%(int(len(disgust[0])/12))==0 and disgust[0][i]!='pornography']
su = [surprise[1][i] for i in range(len(surprise[0])) if i%(int(len(surprise[0])/12))==0]
su1 = [surprise[0][i] for i in range(len(surprise[0])) if i%(int(len(surprise[0])/12))==0]
t = [trust[1][i] for i in range(len(trust[0])) if i%(int(len(trust[0])/12))==0]
t1 = [trust[0][i] for i in range(len(trust[0])) if i%(int(len(trust[0])/12))==0]

In [ ]:
j

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 7))
# fig.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')

ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.99)
# fig.subplots_adjust(bottom=-0.9)
# ax.set_title('axes title')

ax.set_xlabel('Emotion Category',fontsize=12)
ax.set_ylabel('Emotion Intensity', fontsize=12)

# ax.text(3, 8, 'boxed italics text in data coords', style='italic',
        # bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})

# ax.text(2, 6, r'an equation: $E=mc^2$', fontsize=15)

# ax.text(0.95, 0.01, 'colored text in axes coords',
#         verticalalignment='bottom', horizontalalignment='right',
#         transform=ax.transAxes,
#         color='green', fontsize=15)

em=['joy']*len(j)+['fear']*len(f)+ ['anger']*len(a)+ ['sadness']*(len(s)-1)+ ['anticipation']*len(an)+ ['disgust']*len(d)+ ['surprise']*len(su)+ ['trust']*len(t)
emnum = em=[0]*len(j)+[1]*len(f)+ [2]*len(a)+ [3]*(len(s)-1)+ [4]*len(an)+ [5]*len(d)+ [6]*len(su)+ [7]*len(t)
inten = j+f+a+s[:-1]+an+d+su+t
word = j1+f1+a1+s1[:-1]+an1+d1+su1+t1
for i in range(len(em)):
  ax.text(emnum[i]+0.04, inten[i]-0.004, word[i], fontsize=11)

ax.plot(['joy']*len(j) ,j , 'o',color= '#33A7FF')#0000FF)
ax.plot(['fear']*len(f) ,f, 'o',color= '#990012')
ax.plot(['anger']*len(a) ,a, 'o',color= '#E42217')
ax.plot(['sadness']*(len(s)-1) ,s[:-1], 'o',color= '#151B8D')
ax.plot(['anticipation']*len(an) ,an, 'o',color= '#FF8040')
ax.plot(['disgust']*len(d) ,d, 'o',color= '#8E35EF')
ax.plot(['surprise']*len(su) ,su, 'o',color= '#FDD017')
ax.plot(['trust']*len(t) ,t, 'o',color= '#4CC417')
plt.xticks(fontsize=12)
ax.axis([-1, 8, 0, 1])
plt.savefig('emotion-word-dist.pdf', format='pdf', dpi=1200)
plt.show()

In [ ]:
!pip install --upgrade --quiet gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
worksheet = gc.open('Human_Eval_Reponses').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
task1 = pd.DataFrame.from_records(rows)

In [ ]:
task1_aff = task1[:36]
task1_our = task1.iloc[36:]
task1_aff.iloc[-1], task1_our.iloc[0]

In [ ]:
task1_aff.iloc[10][2]

In [ ]:
len(task1_our)

In [ ]:
for j in range(3,8):
  a1=0; a2 =0; a3 = 0; a1c=0; a2c=0; a3c=0
  for i in range(len(task1_aff)):
    if task1_aff.iloc[i][2]=='1':
      a1+=1
      if task1_aff.iloc[i][1] == task1_aff.iloc[i][j]:
        a1c+=1
    if task1_aff.iloc[i][2]=='2':
      a2+=1
      if task1_aff.iloc[i][1] == task1_aff.iloc[i][j]:
        a2c+=1
    if task1_aff.iloc[i][2]=='3':
      a3+=1
      if task1_aff.iloc[i][1] == task1_aff.iloc[i][j]:
        a3c+=1
  print(a1c/a1, a2c/a2, a3c/a3, a1c,a1, a2c,a2, a3c,a3)

In [ ]:
for j in range(3,8):
  a1=0; a2 =0; a3 = 0; a1c=0; a2c=0; a3c=0
  for i in range(len(task1_our)):
    if task1_our.iloc[i][2]=='0.4':
      a1+=1
      if task1_our.iloc[i][1] == task1_our.iloc[i][j]:
        a1c+=1
    if task1_our.iloc[i][2]=='0.6':
      a2+=1
      if task1_our.iloc[i][1] == task1_our.iloc[i][j]:
        a2c+=1
    if task1_our.iloc[i][2]=='1':
      a3+=1
      if task1_our.iloc[i][1] == task1_our.iloc[i][j]:
        a3c+=1
  print(a1c/a1, a2c/a2, a3c/a3, a1c,a1, a2c,a2, a3c,a3)

In [ ]:
task2[:7]

In [ ]:
for j in range(2,6):
  a1=0; a2 =0; a3 = 0; a1c=0; a2c=0; a3c=0
  for i in range(6, len(task2)):
    # print(task2.iloc[i])
    if task2.iloc[i][1]=='0.4':
      a1+=1
      if task2.iloc[i][0] == task2.iloc[i][j]:
        a1c+=1
    if task2.iloc[i][1]=='0.6':
      a2+=1
      if task2.iloc[i][0] == task2.iloc[i][j]:
        a2c+=1
    if task2.iloc[i][1]=='1':
      a3+=1
      if task2.iloc[i][0] == task2.iloc[i][j]:
        a3c+=1
  print(a1c/a1, a2c/a2, a3c/a3, a1c,a1, a2c,a2, a3c,a3)

In [ ]:
!pip install pingouin

In [ ]:
worksheet = gc.open('Human_Eval_Reponses').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
task3 = pd.DataFrame.from_records(rows)

In [ ]:
task3[:16]

In [ ]:
act_int = list(task3[6:6+234][2])
avg = list(task3[6:][8]) ## aditya's-7
print(len(act_int), len(avg))
i=0; gram=[]; pred_int=[]
while i<len(avg):
  gram+=avg[i:i+6]
  pred_int+=avg[i+6:i+12]
  i+=12
len(gram), len(pred_int)

In [ ]:
act_int = [float(i) for i in act_int]
gram = [float(i) for i in gram]
pred_int = [float(i)+1 for i in pred_int]
act_int[:2], gram[:7], pred_int[:7]

In [ ]:
([0.0, 0.6],
 [5.25, 4.5, 4.0, 6.75, 5.75, 6.75, 6.0],
 [3.0, 3.5, 2.75, 3.5, 2.0, 1.25, 1.25])

In [ ]:
import pandas as pd
import pingouin as pg
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
i=0; g= [[] for i in range(13)]; p = [[] for i in range(13)]; j=0
while i <len(act_int):
  data = pd.DataFrame(list(zip(act_int[i:i+18], gram[i:i+18], pred_int[i:i+18])), 
                columns =[ 'act', 'gram', 'pred_int']) 
  
  data = data.sort_values('act')
  g[j].append([np.mean(list(data['gram'])[3*i:3*i+3]) for i in range(6)])
  p[j].append([np.mean(list(data['pred_int'])[3*i:3*i+3]) for i in range(6)])
  # print('gr', g)
  # print('pi', p)
  j+=1
  i+=18

In [ ]:
g[1][0]

In [ ]:


# ax.plot([1, 2])
# ax.set_xlabel('x-label', fontsize=12)
# ax.set_ylabel('y-label', fontsize=12)
# ax.set_title('Title', fontsize=14)

fig2 = plt.figure(constrained_layout=True, figsize=(15, 7))
spec2 = gridspec.GridSpec(ncols=3, nrows=2, figure=fig2)

f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax1.set_xticks([])
f2_ax1.set_title('Our Model', fontsize=14)
f2_ax1.set_ylabel('Perplexity\n (Automated Evaluation)\n (lower is better)', fontsize=12)
perp = [32.92,29.19 ,33.49,30.91,29.37,30.85]
int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
f2_ax1.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
perp= [28.75,29.43,30.09,31.56,30.84,28.58]
f2_ax1.plot(int_, perp,'*-', label = 'Avg. for 8 emotions')
f2_ax1.legend()
f2_ax1.axis([0, 1, 10, 65])
f2_ax1.set_xlabel('(a)', fontsize = 12)


f2_ax2 = fig2.add_subplot(spec2[0, 1])
f2_ax2.set_xticks([])
f2_ax2.set_yticks([])
f2_ax2.set_title('Affect LM', fontsize=14)
perp = [41.01, 36.77,41.84,43.63, 47.86,59.64]
int_ = [0,1,2,3,4,5]
f2_ax2.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
f2_ax2.axis([0, 5, 10, 65])
f2_ax2.legend()
f2_ax2.set_xlabel('(b)', fontsize = 12)


f2_ax5 = fig2.add_subplot(spec2[0, 2])
f2_ax5.set_xticks([])
f2_ax5.set_yticks([])
perp = [14.38,13.13 ,11.95,15.96,64.47,58.76]
int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
f2_ax5.plot(int_, perp,'o-', label = 'Avg. for 2 sentiments')
f2_ax5.axis([0, 1, 10, 65])
f2_ax5.legend()
f2_ax5.set_title('PPLM', fontsize=14)
f2_ax5.set_xlabel('(c)', fontsize = 12)

f2_ax3 = fig2.add_subplot(spec2[1, 0])
int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
f2_ax3.plot(int_, g[0][0],'o-', label = 'Anger')
f2_ax3.plot(int_, g[1][0],'*-', label = 'Sadness')
f2_ax3.plot(int_, g[2][0],'+-', label = 'Joy')
f2_ax3.plot(int_, g[8][0],'x-', label = 'Fear')
f2_ax3.plot(int_, g[9][0],'v-', label = 'Anticipation')
f2_ax3.plot(int_, g[10][0],'D-', label = 'Disgust')
f2_ax3.plot(int_, g[11][0],'d-', label = 'Surprise')
f2_ax3.plot(int_, g[12][0],'^-', label = 'Trust')
f2_ax3.legend()
f2_ax3.set_ylabel('Grammatical Correctness\n (Human Evaluation)\n (higher is better)', fontsize=12)
f2_ax3.axis([0, 1, 1, 7])
f2_ax3.set_xlabel('(d)\nEmotion Intensity (Knob)', fontsize = 12)

f2_ax4 = fig2.add_subplot(spec2[1, 1])
f2_ax4.set_yticks([])
int_ = [0,1,2,3,4,5]
f2_ax4.plot(int_, g[3][0],'o-', label = 'Anger')
f2_ax4.plot(int_, g[4][0],'*-', label = 'Sadness')
f2_ax4.plot(int_, g[5][0],'+-', label = 'Positive Emotion')
f2_ax4.legend()
f2_ax4.axis([0, 5, 1, 7])
f2_ax4.set_xlabel('(e)\nEmotion Intensity (beta)', fontsize = 12)

f2_ax6 = fig2.add_subplot(spec2[1, 2])
f2_ax6.set_yticks([])
int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
f2_ax6.plot(int_, g[6][0],'o-', label = 'Negative Sentiment')
f2_ax6.plot(int_, g[7][0],'*-', label = 'Positive Sentiment')
f2_ax6.legend()
f2_ax6.axis([0, 1, 1,7])
f2_ax6.set_xlabel('(f)\nSentiment Intensity (Stepsize)', fontsize = 12)
fig2.savefig('grammer_2annot.svg', format='svg', dpi=1200)

In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(15, 5))
spec2 = gridspec.GridSpec(ncols=3, nrows=1, figure=fig2)

# f2_ax1 = fig2.add_subplot(spec2[0, 0])
# f2_ax1.set_xticks([])
# f2_ax1.set_title('Our Model', fontsize=14)
# f2_ax1.set_ylabel('Perplexity\n (Automated Evaluation)\n (lower is better)', fontsize=12)
# perp = [32.92,29.19 ,33.49,30.91,29.37,30.85]
# int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
# f2_ax1.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
# perp= [28.75,29.43,30.09,31.56,30.84,28.58]
# f2_ax1.plot(int_, perp,'*-', label = 'Avg. for 8 emotions')
# f2_ax1.legend()
# f2_ax1.axis([0, 1, 10, 65])
# f2_ax1.set_xlabel('(a)', fontsize = 12)


# f2_ax2 = fig2.add_subplot(spec2[0, 1])
# f2_ax2.set_xticks([])
# f2_ax2.set_yticks([])
# f2_ax2.set_title('Affect LM', fontsize=14)
# perp = [41.01, 36.77,41.84,43.63, 47.86,59.64]
# int_ = [0,1,2,3,4,5]
# f2_ax2.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
# f2_ax2.axis([0, 5, 10, 65])
# f2_ax2.legend()
# f2_ax2.set_xlabel('(b)', fontsize = 12)


# f2_ax5 = fig2.add_subplot(spec2[0, 2])
# f2_ax5.set_xticks([])
# f2_ax5.set_yticks([])
# perp = [14.38,13.13 ,11.95,15.96,64.47,58.76]
# int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
# f2_ax5.plot(int_, perp,'o-', label = 'Avg. for 2 sentiments')
# f2_ax5.axis([0, 1, 10, 65])
# f2_ax5.legend()
# f2_ax5.set_title('PPLM', fontsize=14)
# f2_ax5.set_xlabel('(c)', fontsize = 12)

f2_ax3 = fig2.add_subplot(spec2[0, 0])
int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
f2_ax3.plot(int_, p[0][0],'o-', label = 'Anger')
f2_ax3.plot(int_, p[1][0],'*-', label = 'Sadness')
f2_ax3.plot(int_, p[2][0],'+-', label = 'Joy')
f2_ax3.plot(int_, p[8][0],'x-', label = 'Fear')
f2_ax3.plot(int_, p[9][0],'v-', label = 'Anticipation')
f2_ax3.plot(int_, p[10][0],'D-', label = 'Disgust')
f2_ax3.plot(int_, p[11][0],'d-', label = 'Surprise')
f2_ax3.plot(int_, p[12][0],'^-', label = 'Trust')
avg8 = [sum(x)/8 for x in zip(p[0][0], p[1][0],p[2][0],p[8][0],p[9][0],p[10][0],p[11][0],p[12][0])]
f2_ax3.plot(int_, avg8,'s-', label = 'Average of 8 emotions', color="black")
f2_ax3.legend()
f2_ax3.set_title('Our Model', fontsize=14)
f2_ax3.set_ylabel('Emotion Intensity Ranking\n(Human Evaluation)', fontsize=12)
f2_ax3.axis([0, 1, 1, 6])
f2_ax3.set_xlabel('(a)\nEmotion Intensity (Knob)', fontsize = 12)

f2_ax4 = fig2.add_subplot(spec2[0, 1])
f2_ax4.set_yticks([])
int_ = [0,1,2,3,4,5]
f2_ax4.plot(int_, p[3][0],'o-', label = 'Anger')
f2_ax4.plot(int_, p[4][0],'*-', label = 'Sadness')
f2_ax4.plot(int_, p[5][0],'+-', label = 'Positive Emotion')
avg3 = [sum(x)/3 for x in zip(p[3][0], p[4][0],p[5][0])]
f2_ax4.plot(int_, avg3,'s-', label = 'Average of 3 emotions', color="black")
f2_ax4.legend()
f2_ax4.axis([0, 5, 1, 6])
f2_ax4.set_title('Affect LM', fontsize=14)
f2_ax4.set_xlabel('(b)\nEmotion Intensity (beta)', fontsize = 12)

f2_ax6 = fig2.add_subplot(spec2[0, 2])
f2_ax6.set_yticks([])
int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
f2_ax6.plot(int_, p[6][0],'o-', label = 'Negative Sentiment')
f2_ax6.plot(int_, p[7][0],'*-', label = 'Positive Sentiment')
avg2 = [sum(x)/2 for x in zip(p[6][0], p[7][0])]
f2_ax6.plot(int_, avg2,'s-', label = 'Average of 2 sentiments', color="black")
f2_ax6.legend()
f2_ax6.axis([0, 1, 1,6])
f2_ax6.set_title('PPLM', fontsize=14)
f2_ax6.set_xlabel('(c)\nSentiment Intensity (Stepsize)', fontsize = 12)

fig2.savefig('pred_intensity_2annot.svg', format='svg', dpi=1200)

In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(10, 5))
spec2 = gridspec.GridSpec(ncols=2, nrows=1, figure=fig2)

# f2_ax1 = fig2.add_subplot(spec2[0, 0])
# f2_ax1.set_xticks([])
# f2_ax1.set_title('Our Model', fontsize=14)
# f2_ax1.set_ylabel('Perplexity\n (Automated Evaluation)\n (lower is better)', fontsize=12)
# perp = [32.92,29.19 ,33.49,30.91,29.37,30.85]
# int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
# f2_ax1.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
# perp= [28.75,29.43,30.09,31.56,30.84,28.58]
# f2_ax1.plot(int_, perp,'*-', label = 'Avg. for 8 emotions')
# f2_ax1.legend()
# f2_ax1.axis([0, 1, 10, 65])
# f2_ax1.set_xlabel('(a)', fontsize = 12)


# f2_ax2 = fig2.add_subplot(spec2[0, 1])
# f2_ax2.set_xticks([])
# f2_ax2.set_yticks([])
# f2_ax2.set_title('Affect LM', fontsize=14)
# perp = [41.01, 36.77,41.84,43.63, 47.86,59.64]
# int_ = [0,1,2,3,4,5]
# f2_ax2.plot(int_, perp,'o-', label = 'Avg. for 3 emotions')
# f2_ax2.axis([0, 5, 10, 65])
# f2_ax2.legend()
# f2_ax2.set_xlabel('(b)', fontsize = 12)


# f2_ax5 = fig2.add_subplot(spec2[0, 2])
# f2_ax5.set_xticks([])
# f2_ax5.set_yticks([])
# perp = [14.38,13.13 ,11.95,15.96,64.47,58.76]
# int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
# f2_ax5.plot(int_, perp,'o-', label = 'Avg. for 2 sentiments')
# f2_ax5.axis([0, 1, 10, 65])
# f2_ax5.legend()
# f2_ax5.set_title('PPLM', fontsize=14)
# f2_ax5.set_xlabel('(c)', fontsize = 12)

f2_ax3 = fig2.add_subplot(spec2[0, 0])
int_ = [0.0,0.2,0.4,0.6,0.8,1.0]
Ang = [0.451, 0.465, 0.472, 0.473, 0.472, 0.463]
joy = [0.403, 0.409, 0.405, 0.425, 0.426, 0.412]
sad = [0.421, 0.428, 0.428, 0.433, 0.433, 0.420]
f2_ax3.plot(int_, Ang,'o-', label = 'Anger')
f2_ax3.plot(int_, sad,'*-', label = 'Sadness')
f2_ax3.plot(int_, joy,'+-', label = 'Joy')
f2_ax3.legend()
f2_ax3.set_ylabel('Emotion Intensity Rating\n(Automated Evaluation)', fontsize=12)
f2_ax3.axis([0, 1, 0.4, 0.5])
f2_ax3.set_title('Our Model', fontsize=14)
f2_ax3.set_xlabel('(a)\nEmotion Intensity (Knob)', fontsize = 12)

f2_ax4 = fig2.add_subplot(spec2[0, 1])
# f2_ax4.set_yticks([])
int_ = [0,1,2,3,4,5]
pos_em=[0.446, 0.458, 0.501, 0.572, 0.638, 0.621]
sad= [0.403, 0.417, 0.419, 0.431, 0.435, 0.446]
anger = [0.429, 0.435, 0.479, 0.525, 0.557, 0.579]
f2_ax4.plot(int_, anger,'o-', label = 'Anger')
f2_ax4.plot(int_, sad,'*-', label = 'Sadness')
f2_ax4.plot(int_, pos_em,'+-', label = 'Positive Emotion')
f2_ax4.legend()
f2_ax4.set_title('Affect LM', fontsize=14)
f2_ax4.axis([0, 5, 0.4,0.7])
f2_ax4.set_xlabel('(b)\nEmotion Intensity (beta)', fontsize = 12)

# f2_ax6 = fig2.add_subplot(spec2[0, 2])
# f2_ax6.set_yticks([])
# int_ =[0.01,0.02,0.05,0.10,0.50,1.00]
# f2_ax6.plot(int_, p[6][0],'o-', label = 'Negative Sentiment')
# f2_ax6.plot(int_, p[7][0],'*-', label = 'Positive Sentiment')
# f2_ax6.legend()
# f2_ax6.axis([0, 1, 1,6])
# f2_ax6.set_xlabel('(c)\nSentiment Intensity (Stepsize)', fontsize = 12)
fig2.savefig('pred_intensity_auto.svg', format='svg', dpi=1200)

In [ ]:
i=0; m = np.zeros(18); a =0
while i <len(act_int):
  
  data = pd.DataFrame(list(zip(act_int[i:i+18], gram[i:i+18])), 
                columns =[ 'group', 'weight']) 
  # print(data)
  # data = "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/PlantGrowth.csv"

  # df = pd.read_csv(data, index_col=0)
  # data = data.sort_values('group')
  # we = [np.mean(list(data['weight'])[3*i:3*i+3]) for i in range(6)]
  # gr = [np.mean(list(data['group'])[3*i:3*i+3]) for i in range(6)]
  # data1 = pd.DataFrame(list(zip(gr, we)), 
  #               columns =[ 'group', 'weight'])
  # if a in [0,1,2,8,9,10,11,12]:
  #   print(a)
  #   m+=np.array(data['weight'])
  aov = pg.anova(data=data, dv='weight', between='group', detailed=True)
  print(aov)
  # pt = pg.pairwise_tukey(dv='weight', between='group', data=data)
  # print(pt)
  i+=18
  a+=1
data = pd.DataFrame(list(zip(act_int[:18], list(m))), 
                columns =[ 'group', 'weight'])
aov = pg.anova(data=data, dv='weight', between='group', detailed=True)
print(aov)

In [ ]:
0.317, 0.010, 0.048, 0.002, 0.000, 0.0167, 0.352, 0.415, 0.639, 0.203, 0.000, 0.188, 0.067
0.451, 0.146, 0.330, 0.553, -, 1, 0.135, 0.008, 0.749, 0.758, 0.692, 0.625, 0.671

In [ ]:
from statsmodels.multivariate.manova import MANOVA

In [ ]:
i=0
while i <len(act_int):
  data = pd.DataFrame(list(zip(act_int[i:i+18], gram[i:i+18], pred_int[i:i+18])), 
                columns =[ 'group', 'gram', 'pred_int']) 
  maov = MANOVA.from_formula('gram + pred_int ~ group', data=data)
  print(maov.mv_test())
  i+=18

In [ ]:
!pip install krippendorff

In [ ]:
import krippendorff
import numpy as np

In [ ]:
task3[:10]

In [ ]:
reldata1 = []
reldata2 = []
for j in range(4):
  avg = list(task3[6:][j+4]) ## aditya's-7
  print(len(avg))
  i=0; gram=[]; pred_int=[]
  while i<len(avg):
    gram+=avg[i:i+6]
    pred_int+=avg[i+6:i+12]
    i+=12
  gram = [float(i) for i in gram]
  pred_int = [float(i)+1 for i in pred_int]
  reldata1.append(pred_int)
  reldata2.append(gram)
print(len(reldata1))
reliability_data = np.array(reldata1)
reliability_data.reshape(4, 234)
reliability_data.shape

In [ ]:
reliability_data = np.array(reldata2)
reliability_data.reshape(4, 234)
reliability_data.shape
reliability_data[0].shape

In [ ]:
reldata = [list(reliability_data[1]), list(reliability_data[3])]

In [ ]:
krippendorff.alpha(reldata)

In [ ]:
01 -0.09, 02 -0.13, 03 0.08
12 0.47, 13 0.23
23 0.13

In [ ]:
task1 =task1.replace(['Anger'], 0).replace(['Positive Emotion'], 1).replace(['Neutral'], 2).replace(['Sadness'], 3)
task1[-5:]

In [ ]:
reldata = [list(task1[:72][7]), list(task1[:72][6])]#, list(task1[:72][6]), list(task1[:72][7])], 3,4,6,7

In [ ]:
krippendorff.alpha(reldata)

In [ ]:
task2 =task2.replace(['Anger'], 0).replace(['Joy'], 1).replace(['Fear'], 2).replace(['Sadness'], 3).replace(['Disgust'], 4).replace(['Anticipation'], 5).replace(['Surprise'], 6).replace(['Trust'], 7)
task2[:7]

In [ ]:
reldata = [list(task2[6:][4]), list(task2[6:][5])]#, list(task2[6:][4]), list(task2[6:][5])], 2,3,4,5

In [ ]:
krippendorff.alpha(reldata)

In [ ]:
0.804, 0.162 | 
0.36, 43 0.33, 46 0.54, 47 0.29, 37 0.22, 36 0.41, 67 0.31
0.28, 23 0.26, 24 0.40, 25 0.15, 34 0.15, 35 0.42, 45 0.30